In [3]:
import re # expresinones regulares
import requests
from bs4 import BeautifulSoup

In [5]:
contenido_html = requests.get("https://weather.com/es-CO/tiempo/horario/l/9ac15d92af342287ce0421d3daa71cc6e16f8f451fae2a04c524b944242a3933").content
soup = BeautifulSoup(contenido_html, "lxml")

In [6]:
print(soup.find("span", class_="DetailsSummary--tempValue--1K4ka"))

<span class="DetailsSummary--tempValue--1K4ka" data-testid="TemperatureValue">16°</span>


In [11]:
#extrayendo los dias
dias_diccionario ={}
#currentDateId0 el numero despues del "Id" va cambiando
for dia in soup.find_all("h3", attrs={"id":re.compile("^currentDateId")}):
    key=dia.get("id").split("currentDateId")[1]
    value = dia.text # obtener el solo texto del tag por ejemplo : jueves, 7 abril
    #creamos el diccionario
    dias_diccionario[key] = value

In [12]:
dias_diccionario

{'0': 'jueves, 7 de abril',
 '14': 'viernes, 8 de abril',
 '38': 'sábado, 9 de abril'}

In [27]:
datos = {
    "dia": [],
    "hora" : [],
    "temperatura": [],
    "nubosidad" : [],
    "precipitacion" : [],
    "viento" : []
}


# las ventajas de usar un diccionario es para tener la informacion ordenada
#y luego tranformarla en un data frame


In [30]:
# vamos a tomar los datos por la hora en el resumen(summary no existe una id)
for datos_por_hora in soup.find_all("summary", class_="Disclosure--Summary--UuybP DaypartDetails--Summary--3IBUr Disclosure--hideBorderOnSummaryOpen--ZdSDc"):
    key = datos_por_hora.get("aria-labelledby").split()[0].split("currentDateId")[1]
    datos["dia"].append(dias_diccionario[key])
    datos["hora"].append(datos_por_hora.find("h2", attrs={"data-testid":"daypartName"}).text)
    datos["temperatura"].append(datos_por_hora.find("div", attrs={"data-testid":"detailsTemperature"}).text)
    datos["nubosidad"].append(datos_por_hora.find("div", attrs={"data-testid":"wxIcon"}).span.text)
    datos["precipitacion"].append(datos_por_hora.find("div", attrs={"data-testid":"Precip"}).span.text)
    datos["viento"].append(datos_por_hora.find("div", attrs={"data-testid":"wind"}).span.text)

In [31]:
import pandas as pd 

df = pd.DataFrame(datos)
df.head(-1)

,dia,hora,temperatura,nubosidad,precipitacion,viento
0,"jueves, 7 de abril",10:00,16°,Parcialmente nublado,24%,E 4 km/h
1,"jueves, 7 de abril",11:00,16°,Muy nublado,24%,ESE 6 km/h
2,"jueves, 7 de abril",12:00,17°,Nublado,24%,ESE 9 km/h
3,"jueves, 7 de abril",13:00,19°,Nublado,18%,SE 13 km/h
4,"jueves, 7 de abril",14:00,18°,Nublado,18%,SE 13 km/h
...,...,...,...,...,...,...
90,"sábado, 9 de abril",4:00,12°,Muy nublado,18%,NNE 5 km/h
91,"sábado, 9 de abril",5:00,12°,Muy nublado,18%,NE 6 km/h
92,"sábado, 9 de abril",6:00,12°,Muy nublado,18%,NE 6 km/h
93,"sábado, 9 de abril",7:00,13°,Muy nublado,20%,NE 6 km/h
